# OrientedPlane3 Factors

This header defines factors for working with planar landmarks represented by `gtsam.OrientedPlane3`.
`OrientedPlane3` represents a plane using normalized coefficients $(n_x, n_y, n_z, d)$, where $(n_x, n_y, n_z)$ is the unit normal vector and $d$ is the distance from the origin along the normal.

Factors defined:
*   `OrientedPlane3Factor`: A binary factor connecting a `Pose3` and an `OrientedPlane3`. It measures the difference between the plane parameters as observed from the given pose and a measured plane.
*   `OrientedPlane3DirectionPrior`: A unary factor on an `OrientedPlane3`. It penalizes the difference between the plane's normal direction and a measured direction (represented by the normal of a measured `OrientedPlane3`). **Note:** The factor error is 3D, but only constrains 2 degrees of freedom (direction). Consider using a more specific direction factor if only direction is measured.

<a href="https://colab.research.google.com/github/borglab/gtsam/blob/develop/gtsam/slam/doc/OrientedPlane3Factor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install --quiet gtsam-develop

In [2]:
import gtsam
import numpy as np
from gtsam import Pose3, OrientedPlane3, Point3, Rot3, Values
from gtsam import OrientedPlane3Factor, OrientedPlane3DirectionPrior
from gtsam.symbol_shorthand import X, P

## 1. `OrientedPlane3Factor`

Connects a `Pose3` (camera/robot pose) and an `OrientedPlane3` (landmark). The measurement is the plane as observed *from the sensor frame*.
The error is calculated by transforming the global plane landmark into the sensor frame defined by the pose, and then computing the difference (`localCoordinates`) with the measured plane.

In [4]:
# Ground truth plane (e.g., z=1 in world frame)
gt_plane_world = OrientedPlane3(0, 0, 1, 1)

# Ground truth pose
gt_pose = Pose3(Rot3.Yaw(0.1), Point3(0.5, 0, 0))

# Measurement: transform the world plane into the camera frame
# measured_plane = gt_plane_world.transform(gt_pose)
# C++ header: Plane measurement z is a 4D vector [a,b,c,d] coefficients:
measured_plane_coeffs = gt_plane_world.planeCoefficients()
plane_noise = gtsam.noiseModel.Isotropic.Sigma(3, 0.05)

pose_key = X(0)
plane_key = P(0)

plane_factor = OrientedPlane3Factor(measured_plane_coeffs, plane_noise, pose_key, plane_key)
plane_factor.print("OrientedPlane3Factor: ")

# Evaluate error
values = Values()
values.insert(pose_key, gt_pose)
values.insert(plane_key, gt_plane_world)
error1 = plane_factor.error(values)
print(f"\nError at ground truth: {error1}")

# Evaluate with slightly different plane estimate
noisy_plane = OrientedPlane3(0.01, 0.01, 0.99, 1.05)
values.update(plane_key, noisy_plane)
error2 = plane_factor.error(values)
print(f"\nError with noisy plane: {error2}")

OrientedPlane3Factor: 
OrientedPlane3Factor Factor (x0, p0)
Measured Plane : 0 0 1 1
isotropic dim=3 sigma=0.05


TypeError: insert(): incompatible function arguments. The following argument types are supported:
    1. (self: gtsam.gtsam.Values, values: gtsam.gtsam.Values) -> None
    2. (self: gtsam.gtsam.Values, j: int, vector: numpy.ndarray[numpy.float64[m, 1]]) -> None
    3. (self: gtsam.gtsam.Values, j: int, matrix: numpy.ndarray[numpy.float64[m, n]]) -> None
    4. (self: gtsam.gtsam.Values, j: int, point2: numpy.ndarray[numpy.float64[2, 1]]) -> None
    5. (self: gtsam.gtsam.Values, j: int, point3: numpy.ndarray[numpy.float64[3, 1]]) -> None
    6. (self: gtsam.gtsam.Values, j: int, rot2: gtsam.gtsam.Rot2) -> None
    7. (self: gtsam.gtsam.Values, j: int, pose2: gtsam.gtsam.Pose2) -> None
    8. (self: gtsam.gtsam.Values, j: int, R: gtsam.gtsam.SO3) -> None
    9. (self: gtsam.gtsam.Values, j: int, Q: gtsam.gtsam.SO4) -> None
    10. (self: gtsam.gtsam.Values, j: int, P: gtsam.gtsam.SOn) -> None
    11. (self: gtsam.gtsam.Values, j: int, rot3: gtsam.gtsam.Rot3) -> None
    12. (self: gtsam

## 2. `OrientedPlane3DirectionPrior`

A unary factor that puts a prior on the *direction* (normal vector) of an `OrientedPlane3`. The distance component (`d`) of the measured plane is ignored.
The error is the 3D difference between the estimated plane's normal and the measured plane's normal.

In [6]:
# Measured direction prior (e.g., plane normal is close to world Z axis)
measured_prior_plane = OrientedPlane3(0, 0, 1, 0) # Distance (last coeff) is ignored
direction_noise = gtsam.noiseModel.Isotropic.Sigma(3, 0.02)

prior_factor = OrientedPlane3DirectionPrior(plane_key, measured_prior_plane.planeCoefficients(), direction_noise)
prior_factor.print("OrientedPlane3DirectionPrior: ")

# Evaluate error using the 'noisy_plane' from the previous step
error_prior = prior_factor.error(values) # values still contains plane_key -> noisy_plane
print(f"\nError for prior on noisy_plane: {error_prior}")

# Evaluate error for ground truth plane
values.update(plane_key, gt_plane_world)
error_prior_gt = prior_factor.error(values)
print(f"Error for prior on gt_plane_world: {error_prior_gt}")

OrientedPlane3DirectionPrior: 
OrientedPlane3DirectionPrior: Prior Factor on p0
Measured Plane : 0 0 1 0
isotropic dim=3 sigma=0.02


RuntimeError: Attempting to at the key "p0", which does not exist in the Values.